### Arhum Zafar
<br>
<br>
A brief attempt of using reinforcement learning strategies to make a quick trading agent.

In [52]:
# %watermark

In [4]:
#!pip install tensorflow-gpu==2.0.0.alpha0|
from src.config.config import Config
from src.db_writer.db import DB

### Dependencies

In [5]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [6]:
tf.__version__

'2.9.0'

In [7]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8552659872410419350
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6932135936
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15151203759784713500
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


#### Building the trading network

In [8]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=16, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state, verbose=0)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0])
        
      target = self.model.predict(state, verbose=0)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

#### Preprocessing

In [9]:
#Sigmoid
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#Price Format Function
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Load Dataset

In [10]:
def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  config = Config()
  db = DB(config)
  funding_data = pd.read_sql('SELECT * FROM `binance`.funding_data;', con=db.con)
  funding_data = funding_data.sort_values(by='timestamp')
  funding_data

  # dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  # start_date = str(dataset.index[0]).split()[0]
  # end_date = str(dataset.index[-1]).split()[0]
  
  close = funding_data['mark_price']
  return close

#### State Creator

In [11]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  # print(starting_id)
  # print(data[0])
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
  windowed_data = list(windowed_data)
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

#### Loading a dataset for a stock

In [12]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

### Training

In [17]:
# hyperparameters

window_size = 600
episodes = 100

batch_size = 32
data_samples = len(data) - 1 - window_size

In [18]:
trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                19232     
                                                                 
 dense_5 (Dense)             (None, 64)                2112      
                                                                 
 dense_6 (Dense)             (None, 128)               8320      
                                                                 
 dense_7 (Dense)             (None, 3)                 387       
                                                                 
Total params: 30,051
Trainable params: 30,051
Non-trainable params: 0
_________________________________________________________________


In [19]:

with tf.device(tf.DeviceSpec(device_type="GPU")):
  for episode in range(1, episodes + 1):
    
    print("Episode: {}/{}".format(episode, episodes))
    
    state = state_creator(data, 0, window_size + 1)
    
    total_profit = 0
    trader.inventory = []
    
    for t in tqdm(range(data_samples)):
      
      action = trader.trade(state)
      # print((t+1, window_size + 1))
      next_state = state_creator(data, t+1, window_size + 1)
      reward = 0
      
      if action == 1: #Buying
        trader.inventory.append(data[t])
        print("AI Trader bought: ", stocks_price_format(data[t]))
        
      elif action == 2 and len(trader.inventory) > 0: #Selling
        buy_price = trader.inventory.pop(0)
        
        reward = max(data[t] - buy_price, 0)
        total_profit += data[t] - buy_price
        print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
        
      if t == data_samples - 1:
        done = True
      else:
        done = False
        
      trader.memory.append((state, action, reward, next_state, done))
      
      state = next_state
      
      if done:
        print("########################")
        print("TOTAL PROFIT: {}".format(total_profit))
        print("########################")
      
      if len(trader.memory) > batch_size:
        trader.batch_train(batch_size)
        
    if episode % 10 == 0:
      trader.model.save("ai_trader_{}.h5".format(episode))
        

Episode: 1/100


  0%|          | 0/116407 [00:00<?, ?it/s]

AI Trader bought:  $ 22418.800000
AI Trader bought:  $ 22417.000000
AI Trader bought:  $ 22412.100000
AI Trader bought:  $ 22417.800000
AI Trader bought:  $ 22421.600000
AI Trader sold:  $ 22339.300000  Profit: - $ 79.500000
AI Trader bought:  $ 22343.200000
AI Trader bought:  $ 22347.800000
AI Trader bought:  $ 22346.500000
AI Trader bought:  $ 22347.200000
AI Trader bought:  $ 22342.000000
AI Trader bought:  $ 22345.400000
AI Trader sold:  $ 22345.400000  Profit: - $ 71.600000
AI Trader sold:  $ 22345.500000  Profit: - $ 66.600000
AI Trader bought:  $ 22345.500000
AI Trader sold:  $ 22336.900000  Profit: - $ 80.900000
AI Trader bought:  $ 22335.900000
AI Trader bought:  $ 22338.000000
AI Trader bought:  $ 22341.400000
AI Trader bought:  $ 22340.100000
AI Trader sold:  $ 22338.700000  Profit: - $ 82.900000
AI Trader bought:  $ 22336.900000
AI Trader bought:  $ 22334.500000
AI Trader sold:  $ 22337.100000  Profit: - $ 6.100000
AI Trader bought:  $ 22336.400000
AI Trader bought:  $ 2233

  0%|          | 33/116407 [00:03<3:49:52,  8.44it/s]

AI Trader sold:  $ 22330.800000  Profit: - $ 17.000000


...

AI Trader bought:  $ 22298.800000


  1%|          | 749/116407 [59:54<142:17:38,  4.43s/it]

AI Trader sold:  $ 22297.500000  Profit: $ 17.600000


  1%|          | 750/116407 [59:58<135:21:04,  4.21s/it]

AI Trader sold:  $ 22328.500000  Profit: $ 48.100000


Depending on whether you have a GPU or not, the loop time will vary. As the loop iterates, you will notice how the network makes stronger predictions over time; reinforcing its behavior over while iterating, yielding greater profit.